## GLIMS region files download: Level 0 files

This script creates "Level 0" RGI files. These files are fetched directly from the GLIMS database, subsetted with a spatial bounding box around each region.

Level 0 files need to be updated to reflect new entries into the GLIMS database.

In [ ]:
from glims_database_dump import *
import geopandas as gpd
import shutil
from utils import mkdir
import numpy as np

In [ ]:
servers = {
        'production': 'www.glims.org/services',
        'blue': 'blue.glims-services.apps.int.nsidc.org',
        'integration': 'integration.glims-services.apps.int.nsidc.org',
        'qa': 'qa.glims-services.apps.int.nsidc.org',
        'staging': 'staging.glims-services.apps.int.nsidc.org',
        }

## RGI Region files 

In [ ]:
# go down from rgi7_scripts/workflow/preprocessing
data_dir = '../../../rgi7_data/'

In [ ]:
reg_file = os.path.join(data_dir, 'l0_regions', '00_rgi70_regions', '00_rgi70_O1Regions.shp')
reg_f = gpd.read_file(reg_file)
reg_f.plot(alpha=0.5, edgecolor='k');

These are the two regions with more than one box Region 01 and region 10:

In [ ]:
reg_f.loc[[0, 1]].plot(alpha=0.5, edgecolor='k');

In [ ]:
# We drop the second alaska box which is useless
reg_f = reg_f.drop(1)

In [ ]:
reg_f.loc[[0]].plot(alpha=0.5, edgecolor='k');

The Alaska box 1 has no glaciers in GLIMS to date (07.06.2021), you'll need to remove it otherwize the download will stall.

Region 10 has two boxes with glaciers in it:

In [ ]:
reg_f.loc[[10, 11]].plot(alpha=0.5, edgecolor='k');

## Download loop 

In [ ]:
# If you want to (re-)download only selected regions
reg_f = reg_f.loc[[5]]
reg_f

In [ ]:
buffer = 0.5  # in degrees, buffer around the box
from_glims = mkdir(os.path.join(data_dir, 'l0_from_glims'))

for i, reg in reg_f.iterrows():
    
    # Prepare bounds 
    x0, y0, x1, y1 = reg.geometry.bounds
    
    x0 = np.clip(x0 - buffer, -180., None)
    y0 = np.clip(y0 - buffer, -90., None)
    x1 = np.clip(x1 + buffer, None, 180.)
    y1 = np.clip(y1 + buffer, None, 90.)
    
    bounds = f' {x0:.2f},{y0:.2f},{x1:.2f},{y1:.2f}'
    print('')
    print('{}, {}. Bounds: {}'.format(reg.RGI_CODE, reg.FULL_NAME, bounds))
        
    # GLIMS request arguments
    p = setup_argument_parser()
    args = p.parse_args(['--mode', 'glims',
                         '--nunataks', 'GLIMS',  # We can't use GLIMS because it is buggy 
                         '--archive_type', 'tar',
                         '--download_type', 'extent',
                         '--clause', bounds])

    server = servers[args.env]

    filebasename = issue_order(server, args)
    poll_readiness(server, filebasename, period=2, tries=args.tries, protocol=args.protocol)    # returns when file is ready
    do_download(server, filebasename, args)
    
    shutil.move(filebasename, os.path.join(from_glims, '{:02d}_RGI{:02d}.tgz'.format(i, int(reg['RGI_CODE']))))